In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
!nvidia-smi

Wed Apr  3 03:53:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   26C    P0              56W / 400W |  31300MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!pip install scikit-learn  -U

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd

def get_eksperiment_baseline(df_train, df_test, fitur, bencana):
    %cd ..
    print(f"Eksperimen pada bencana {bencana} dengan representasi teks {fitur}")
    dct_cls = {}
    vectorizer = CountVectorizer(ngram_range=(1,2))
    LABEL2INDEX = { 'N':0,  'Y':1}
    y_train = list(df_train['label'].apply(lambda lab: LABEL2INDEX[lab]))
    y_test = list(df_test['label'].apply(lambda lab: LABEL2INDEX[lab]))
    # df_all = pd.concat([df_train, df_test], ignore_index=True)
    X_train = vectorizer.fit_transform(df_train[fitur].values.astype('U'))
    X_test = vectorizer.transform(df_test[fitur].values.astype('U')) 

    rf = RandomForestClassifier(max_depth=100, n_estimators=100)
    lr = LogisticRegression(max_iter = 100)
    gnb = MultinomialNB()
    mlp = MLPClassifier(random_state=1, max_iter=300)

    print("RANDOM FOREST")
    rf.fit(X_train, y_train)

    y_pred_train = rf.predict(X_train)
    y_pred_test = rf.predict(X_test)
    print("Di train dapat performa ")
    get_eval(y_pred_train, y_train)
    print("Di test dapat performa")
    p,r,f = get_eval(y_pred_test, y_test)
    dct_metric = {}
    dct_metric["prec"] = p
    dct_metric["rec"] = r
    dct_metric["f1"] = f 
    dct_metric["acc"] = accuracy_score(y_pred_test, y_test)
    dct_cls["rf"] = dct_metric
    print("########")
    df = df_test[['id']]
    df['label'] = y_pred_test
    df[['id','label']].to_csv(f'../prediction/result_classification_Baseline/{bencana}_{fitur}_RF.csv', index=False)

    print("LOGISTIK REGRESSION")
    lr.fit(X_train, y_train)
    y_pred_train = lr.predict(X_train)
    y_pred_test = lr.predict(X_test)
    print("Di train dapat performa ")
    get_eval(y_pred_train, y_train)
    print("Di test dapat performa")
    p_l, r_l, f_l = get_eval(y_pred_test, y_test)
    dct_metric = {}
    dct_metric["prec"] = p_l
    dct_metric["rec"] = r_l
    dct_metric["f1"] = f_l 
    dct_metric["acc"] = accuracy_score(y_pred_test, y_test)
    dct_cls["lr"] = dct_metric
    print("########")
    df = df_test[['id']]
    df['label'] = y_pred_test
    df[['id','label']].to_csv(f'../prediction/result_classification_Baseline/{bencana}_{fitur}_LR.csv', index=False)
    p+=p_l
    r+=r_l
    f+=f_l

    print("NAIVE BAYES")
    X_train.toarray()
    X_test.toarray()
    gnb.fit(X_train, y_train)
    y_pred_train = gnb.predict(X_train)
    y_pred_test = gnb.predict(X_test)
    print("Di train dapat performa ")
    get_eval(y_pred_train, y_train)
    print("Di test dapat performa")
    p_l, r_l, f_l = get_eval(y_pred_test, y_test)
    dct_metric = {}
    dct_metric["prec"] = p_l
    dct_metric["rec"] = r_l
    dct_metric["f1"] = f_l 
    dct_metric["acc"] = accuracy_score(y_pred_test, y_test)
    dct_cls["mnb"] = dct_metric
    print("########")
    df = df_test[['id']]
    df['label'] = y_pred_test
    df[['id','label']].to_csv(f'../prediction/result_classification_Baseline/{bencana}_{fitur}_NB.csv', index=False)
    p+=p_l
    r+=r_l
    f+=f_l

    print("MLP")
    mlp.fit(X_train, y_train)
    y_pred_train = mlp.predict(X_train)
    y_pred_test = mlp.predict(X_test)
    print("Di train dapat performa ")
    get_eval(y_pred_train, y_train)
    print("Di test dapat performa")
    p_l, r_l, f_l = get_eval(y_pred_test, y_test)
    dct_metric = {}
    dct_metric["prec"] = p_l
    dct_metric["rec"] = r_l
    dct_metric["f1"] = f_l 
    dct_metric["acc"] = accuracy_score(y_pred_test, y_test)
    dct_cls["mlp"] = dct_metric
    print("########")
    df = df_test[['id']]
    df['label'] = y_pred_test
    df[['id','label']].to_csv(f'../prediction/result_classification_Baseline/{bencana}_{fitur}_MLP.csv', index=False)
    # p+=p_l
    # r+=r_l
    # f+=f_l
    %cd -
    print('Rata-rata nilai test dengan Classifier rf, lr, dan nb adalah')
    print(f'Precision = {p/3}\nRecall = {r/3}\nF1 = {f/3}')
    return dct_cls

def get_eval(ypred, yact):
    p,r, f,s = precision_recall_fscore_support(yact,ypred, average='binary', pos_label=1)
    acc = accuracy_score(yact, ypred)
    print(f"Nilai Precision = {p}")
    print(f"Nilai Recall = {r}")
    print(f"Nilai F1 = {f}")
    return p,r,f
def get_f1_baseline_bencana(bencana):
    lst_fitur = ["id","title_ori", "text_ori", "tc_ori", "filter_word_tc","filter_word_tc_window", "trunc_first_short", "trunc_last_short", "trunc_first_long", "trunc_last_long", "trunc_combination",  "textrank", "sum_indobart", "sum_liputan6","sum_pegasus","label"]

    df_train = pd.read_csv(f'{bencana}_train.csv')[lst_fitur]# angin_topan_dev.csv
    df_dev = pd.read_csv(f'{bencana}_dev.csv')[lst_fitur]
    df_test = pd.read_csv(f'{bencana}_test.csv')[lst_fitur]
    #df_train_all = pd.concat([df_train, df_dev], ignore_index=True)
    #print(len(df_train),len(df_dev),len(df_test))
    print(f"evaluasi pada bencana = {bencana}")

    dct_bencana={}

    dct_bencana[bencana] = {}
    for fitur in lst_fitur[1:-1]:
        dct_fitur = {}
        dct_bencana[bencana][fitur] = get_eksperiment_baseline(df_train, df_test, fitur, bencana)
        print('\n')
    

    print('************************************************************')


    print(dct_bencana)

In [4]:
!pwd
%cd dataset

/workspace/Tugas Akhir
/workspace/Tugas Akhir/dataset


In [5]:
%cd angin_topan
get_f1_baseline_bencana('angin_topan')
%cd ..

/workspace/Tugas Akhir/dataset/angin_topan
evaluasi pada bencana = angin_topan
/workspace/Tugas Akhir/dataset
Eksperimen pada bencana angin_topan dengan representasi teks title_ori
RANDOM FOREST
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.9259259259259259
Nilai Recall = 0.5434782608695652
Nilai F1 = 0.6849315068493151
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.9874213836477987
Nilai F1 = 0.9936708860759493
Di test dapat performa
Nilai Precision = 0.9696969696969697
Nilai Recall = 0.6956521739130435
Nilai F1 = 0.810126582278481
########
NAIVE BAYES
Di train dapat performa 
Nilai Precision = 0.8863636363636364
Nilai Recall = 0.9811320754716981
Nilai F1 = 0.9313432835820894
Di test dapat performa
Nilai Precision = 0.9285714285714286
Nilai Recall = 0.8478260869565217
Nilai F1 = 0.8863636363636365
########
MLP


/tmp/ipykernel_79270/1540884036.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_79270/1540884036.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_79270/1540884036.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.9230769230769231
Nilai Recall = 0.782608695652174
Nilai F1 = 0.8470588235294118
########
/workspace/Tugas Akhir/dataset/angin_topan
Rata-rata nilai test dengan Classifier rf, lr, dan nb adalah
Precision = 0.9413981080647748
Recall = 0.6956521739130435
F1 = 0.7938072418304776


/workspace/Tugas Akhir/dataset
Eksperimen pada bencana angin_topan dengan representasi teks text_ori


/tmp/ipykernel_79270/1540884036.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


RANDOM FOREST
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 1.0
Nilai Recall = 0.21739130434782608
Nilai F1 = 0.3571428571428571
########
LOGISTIK REGRESSION


...

/tmp/ipykernel_79270/1540884036.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.8333333333333334
Nilai Recall = 0.21739130434782608
Nilai F1 = 0.3448275862068966
########
NAIVE BAYES
Di train dapat performa 
Nilai Precision = 0.9753086419753086
Nilai Recall = 0.9937106918238994
Nilai F1 = 0.984423676012461
Di test dapat performa
Nilai Precision = 0.9090909090909091
Nilai Recall = 0.43478260869565216
Nilai F1 = 0.5882352941176471
########
MLP


/tmp/ipykernel_79270/1540884036.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_79270/1540884036.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.8260869565217391
Nilai Recall = 0.41304347826086957
Nilai F1 = 0.5507246376811594
########
/workspace/Tugas Akhir/dataset/angin_topan
Rata-rata nilai test dengan Classifier rf, lr, dan nb adalah
Precision = 0.8308080808080809
Recall = 0.2608695652173913
F1 = 0.38509503418225527


************************************************************
{'angin_topan': {'title_ori': {'rf': {'prec': 0.9259259259259259, 'rec': 0.5434782608695652, 'f1': 0.6849315068493151, 'acc': 0.885}, 'lr': {'prec': 0.9696969696969697, 'rec': 0.6956521739130435, 'f1': 0.810126582278481, 'acc': 0.925}, 'mnb': {'prec': 0.9285714285714286, 'rec': 0.8478260869565217, 'f1': 0.8863636363636365, 'acc': 0.95}, 'mlp': {'prec': 0.9230769230769231, 'rec': 0.782608695652174, 'f1': 0.8470588235294118, 'acc': 0.935}}, 'text_ori': {'rf': {'prec': 1.0, 'rec': 0.21739130434782608, 'f1': 0.3571428571428571, 'acc

/tmp/ipykernel_79270/1540884036.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


In [ ]:
%cd banjir
get_f1_baseline_bencana('banjir')
%cd ..

/workspace/dataset/banjir
evaluasi pada bencana = banjir
/workspace/dataset
Eksperimen pada bencana banjir dengan representasi teks title_ori
RANDOM FOREST
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.995475113122172
Nilai F1 = 0.9977324263038548
Di test dapat performa
Nilai Precision = 0.7391304347826086
Nilai Recall = 0.3148148148148148
Nilai F1 = 0.4415584415584415
########
LOGISTIK REGRESSION


/tmp/ipykernel_2101906/3747505018.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.995475113122172
Nilai F1 = 0.9977324263038548
Di test dapat performa
Nilai Precision = 0.8108108108108109
Nilai Recall = 0.5555555555555556
Nilai F1 = 0.6593406593406594
########
NAIVE BAYES
Di train dapat performa 
Nilai Precision = 0.9650655021834061

/tmp/ipykernel_2101906/3747505018.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_2101906/3747505018.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test



Nilai Recall = 1.0
Nilai F1 = 0.9822222222222223
Di test dapat performa
Nilai Precision = 0.7291666666666666
Nilai Recall = 0.6481481481481481
Nilai F1 = 0.6862745098039215
########
MLP
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.7631578947368421
Nilai Recall = 0.5370370370370371
Nilai F1 = 0.6304347826086957
########
/workspace/dataset/banjir
Rata-rata nilai test dengan Classifier rf, lr, dan nb adalah
Precision = 0.7597026374200286
Recall = 0.5061728395061729
F1 = 0.5957245369010075


/workspace/dataset
Eksperimen pada bencana banjir dengan representasi teks text_ori


...

/tmp/ipykernel_2101906/3747505018.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.7777777777777778
Nilai Recall = 0.6481481481481481
Nilai F1 = 0.707070707070707
########
NAIVE BAYES


/tmp/ipykernel_2101906/3747505018.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.9778761061946902
Nilai Recall = 1.0
Nilai F1 = 0.9888143176733781
Di test dapat performa
Nilai Precision = 0.676056338028169
Nilai Recall = 0.8888888888888888
Nilai F1 = 0.7679999999999999
########
MLP


/tmp/ipykernel_2101906/3747505018.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


In [ ]:
%cd erupsi
get_f1_baseline_bencana('erupsi')
%cd ..

In [ ]:
%cd gempa
get_f1_baseline_bencana('gempa')
%cd ..

In [ ]:
%cd karhutla
get_f1_baseline_bencana('karhutla')
%cd ..

In [ ]:
%cd kekeringan
get_f1_baseline_bencana('kekeringan')
%cd ..

In [4]:
%cd longsor
get_f1_baseline_bencana('longsor')
%cd ..

/workspace/dataset/longsor
evaluasi pada bencana = longsor
/workspace/dataset
Eksperimen pada bencana longsor dengan representasi teks title_ori
RANDOM FOREST
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.75
Nilai Recall = 0.33962264150943394
Nilai F1 = 0.4675324675324676
########
LOGISTIK REGRESSION


/tmp/ipykernel_1571/3747505018.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.9907834101382489
Nilai F1 = 0.9953703703703703
Di test dapat performa
Nilai Precision = 0.7674418604651163
Nilai Recall = 0.6226415094339622
Nilai F1 = 0.6875
########
NAIVE BAYES


/tmp/ipykernel_1571/3747505018.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.9388646288209607
Nilai Recall = 0.9907834101382489
Nilai F1 = 0.9641255605381167
Di test dapat performa
Nilai Precision = 0.7804878048780488
Nilai Recall = 0.6037735849056604
Nilai F1 = 0.6808510638297872
########
MLP


...

/tmp/ipykernel_1571/3747505018.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 0.9862385321100917
Nilai Recall = 0.9907834101382489
Nilai F1 = 0.9885057471264368
Di test dapat performa
Nilai Precision = 0.5806451612903226
Nilai Recall = 0.33962264150943394
Nilai F1 = 0.42857142857142855
########
MLP


/tmp/ipykernel_1571/3747505018.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.5277777777777778
Nilai Recall = 0.3584905660377358
Nilai F1 = 0.42696629213483145
########
/workspace/dataset/longsor
Rata-rata nilai test dengan Classifier rf, lr, dan nb adalah
Precision = 0.5840245775729646
Recall = 0.23270440251572325
F1 = 0.3155067155067155


************************************************************
{'longsor': {'title_ori': {'rf': {'prec': 0.75, 'rec': 0.33962264150943394, 'f1': 0.4675324675324676, 'acc': 0.795}, 'lr': {'prec': 0.7674418604651163, 'rec': 0.6226415094339622, 'f1': 0.6875, 'acc': 0.85}, 'mnb': {'prec': 0.7804878048780488, 'rec': 0.6037735849056604, 'f1': 0.6808510638297872, 'acc': 0.85}, 'mlp': {'prec': 0.7948717948717948, 'rec': 0.5849056603773585, 'f1': 0.6739130434782609, 'acc': 0.85}}, 'text_ori': {'rf': {'prec': 0.75, 'rec': 0.11320754716981132, 'f1': 0.19672131147540986, 'acc': 0.755}, 'lr': {'prec': 0.693877551020408

/tmp/ipykernel_1571/3747505018.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


In [5]:
%cd tsunami
get_f1_baseline_bencana('tsunami')
%cd ..

/workspace/dataset/tsunami
evaluasi pada bencana = tsunami
/workspace/dataset
Eksperimen pada bencana tsunami dengan representasi teks title_ori
RANDOM FOREST
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.9956331877729258
Nilai F1 = 0.9978118161925601
Di test dapat performa
Nilai Precision = 0.7142857142857143
Nilai Recall = 0.5208333333333334
Nilai F1 = 0.6024096385542168
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 0.9912663755458515
Nilai F1 = 0.9956140350877193
Di test dapat performa
Nilai Precision = 0.7560975609756098
Nilai Recall = 0.6458333333333334
Nilai F1 = 0.6966292134831461
########
NAIVE BAYES
Di train dapat performa 
Nilai Precision = 0.9196787148594378
Nilai Recall = 1.0
Nilai F1 = 0.9581589958158996
Di test dapat performa
Nilai Precision = 0.6491228070175439
Nilai Recall = 0.7708333333333334
Nilai F1 = 0.7047619047619048
########
MLP


/tmp/ipykernel_1571/3747505018.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_1571/3747505018.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_1571/3747505018.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.6595744680851063
Nilai Recall = 0.6458333333333334
Nilai F1 = 0.6526315789473683
########
/workspace/dataset/tsunami
Rata-rata nilai test dengan Classifier rf, lr, dan nb adalah
Precision = 0.7065020274262892
Recall = 0.6458333333333334
F1 = 0.6679335855997559


/workspace/dataset
Eksperimen pada bencana tsunami dengan representasi teks text_ori


/tmp/ipykernel_1571/3747505018.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


RANDOM FOREST
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.8421052631578947
Nilai Recall = 0.3333333333333333
Nilai F1 = 0.4776119402985075
########
LOGISTIK REGRESSION


...

/tmp/ipykernel_1571/3747505018.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test


Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.7391304347826086
Nilai Recall = 0.3541666666666667
Nilai F1 = 0.4788732394366197
########
LOGISTIK REGRESSION
Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.696969696969697
Nilai Recall = 0.4791666666666667
Nilai F1 = 0.5679012345679013
########
NAIVE BAYES
Di train dapat performa 
Nilai Precision = 0.9613733905579399
Nilai Recall = 0.9781659388646288
Nilai F1 = 0.9696969696969697
Di test dapat performa
Nilai Precision = 0.6808510638297872
Nilai Recall = 0.6666666666666666
Nilai F1 = 0.6736842105263158
########
MLP


/tmp/ipykernel_1571/3747505018.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_1571/3747505018.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
/tmp/ipykernel_1571/3747505018.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

Di train dapat performa 
Nilai Precision = 1.0
Nilai Recall = 1.0
Nilai F1 = 1.0
Di test dapat performa
Nilai Precision = 0.6458333333333334
Nilai Recall = 0.6458333333333334
Nilai F1 = 0.6458333333333334
########
/workspace/dataset/tsunami
Rata-rata nilai test dengan Classifier rf, lr, dan nb adalah
Precision = 0.7056503985273643
Recall = 0.5
F1 = 0.5734862281769456


************************************************************
{'tsunami': {'title_ori': {'rf': {'prec': 0.7142857142857143, 'rec': 0.5208333333333334, 'f1': 0.6024096385542168, 'acc': 0.835}, 'lr': {'prec': 0.7560975609756098, 'rec': 0.6458333333333334, 'f1': 0.6966292134831461, 'acc': 0.865}, 'mnb': {'prec': 0.6491228070175439, 'rec': 0.7708333333333334, 'f1': 0.7047619047619048, 'acc': 0.845}, 'mlp': {'prec': 0.6595744680851063, 'rec': 0.6458333333333334, 'f1': 0.6526315789473683, 'acc': 0.835}}, 'text_ori': {'rf': {'prec': 0.8421052631578947, 'rec': 0.3333333333333333, 'f1': 0.4776119402985075, 'acc': 0.825}, 'lr': {'p

/tmp/ipykernel_1571/3747505018.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = y_pred_test
